# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 1:02PM,237875,10,8053092,"Snap Medical Industries, LLC",Released
1,May 23 2022 1:02PM,237875,10,8053093,"Snap Medical Industries, LLC",Released
2,May 23 2022 1:02PM,237875,10,8053094,"Snap Medical Industries, LLC",Released
3,May 23 2022 1:02PM,237875,10,8053095,"Snap Medical Industries, LLC",Released
4,May 23 2022 1:02PM,237875,10,8053096,"Snap Medical Industries, LLC",Released
5,May 23 2022 1:02PM,237875,10,8053097,"Snap Medical Industries, LLC",Released
6,May 23 2022 1:02PM,237875,10,8053098,"Snap Medical Industries, LLC",Released
7,May 23 2022 1:02PM,237875,10,8053099,"Snap Medical Industries, LLC",Released
8,May 23 2022 1:02PM,237875,10,8053100,"Snap Medical Industries, LLC",Released
9,May 23 2022 1:02PM,237875,10,8053101,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,237870,Released,8
34,237871,Released,1
35,237872,Released,7
36,237874,Released,1
37,237875,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237870,NaN,NaN,8.0
237871,NaN,NaN,1.0
237872,NaN,NaN,7.0
237874,NaN,NaN,1.0
237875,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237794,0.0,10.0,16.0
237795,0.0,0.0,1.0
237796,0.0,0.0,1.0
237804,11.0,3.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237794,0,10,16
237795,0,0,1
237796,0,0,1
237804,11,3,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237794,0,10,16
2,237795,0,0,1
3,237796,0,0,1
4,237804,11,3,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237794,,10,16
2,237795,,,1
3,237796,,,1
4,237804,11,3,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC"
13,May 23 2022 12:54PM,237874,19,ACG North America LLC
14,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd"
21,May 23 2022 12:39PM,237871,10,MedStone Pharma LLC
22,May 23 2022 12:36PM,237870,10,"Snap Medical Industries, LLC"
30,May 23 2022 12:34PM,237869,10,ISDIN Corporation
38,May 23 2022 12:25PM,237867,22,"NBTY Global, Inc."
39,May 23 2022 12:24PM,237866,22,"NBTY Global, Inc."
40,May 23 2022 12:23PM,237863,10,Bio-PRF
57,May 23 2022 12:20PM,237673,18,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC",,,13
1,May 23 2022 12:54PM,237874,19,ACG North America LLC,,,1
2,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd",,,7
3,May 23 2022 12:39PM,237871,10,MedStone Pharma LLC,,,1
4,May 23 2022 12:36PM,237870,10,"Snap Medical Industries, LLC",,,8
5,May 23 2022 12:34PM,237869,10,ISDIN Corporation,,,8
6,May 23 2022 12:25PM,237867,22,"NBTY Global, Inc.",,,1
7,May 23 2022 12:24PM,237866,22,"NBTY Global, Inc.",,,1
8,May 23 2022 12:23PM,237863,10,Bio-PRF,,,17
9,May 23 2022 12:20PM,237673,18,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC",13,,
1,May 23 2022 12:54PM,237874,19,ACG North America LLC,1,,
2,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd",7,,
3,May 23 2022 12:39PM,237871,10,MedStone Pharma LLC,1,,
4,May 23 2022 12:36PM,237870,10,"Snap Medical Industries, LLC",8,,
5,May 23 2022 12:34PM,237869,10,ISDIN Corporation,8,,
6,May 23 2022 12:25PM,237867,22,"NBTY Global, Inc.",1,,
7,May 23 2022 12:24PM,237866,22,"NBTY Global, Inc.",1,,
8,May 23 2022 12:23PM,237863,10,Bio-PRF,17,,
9,May 23 2022 12:20PM,237673,18,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC",13,,
1,May 23 2022 12:54PM,237874,19,ACG North America LLC,1,,
2,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd",7,,
3,May 23 2022 12:39PM,237871,10,MedStone Pharma LLC,1,,
4,May 23 2022 12:36PM,237870,10,"Snap Medical Industries, LLC",8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC",13.0,NaN,NaN
1,May 23 2022 12:54PM,237874,19,ACG North America LLC,1.0,NaN,NaN
2,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd",7.0,NaN,NaN
3,May 23 2022 12:39PM,237871,10,MedStone Pharma LLC,1.0,NaN,NaN
4,May 23 2022 12:36PM,237870,10,"Snap Medical Industries, LLC",8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC",13.0,0.0,0.0
1,May 23 2022 12:54PM,237874,19,ACG North America LLC,1.0,0.0,0.0
2,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd",7.0,0.0,0.0
3,May 23 2022 12:39PM,237871,10,MedStone Pharma LLC,1.0,0.0,0.0
4,May 23 2022 12:36PM,237870,10,"Snap Medical Industries, LLC",8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3091957,325.0,13.0,11.0
12,713554,3.0,1.0,0.0
15,237794,16.0,10.0,0.0
18,1664541,7.0,0.0,0.0
19,713598,9.0,0.0,0.0
20,713563,60.0,20.0,1.0
22,475733,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3091957,325.0,13.0,11.0
1,12,713554,3.0,1.0,0.0
2,15,237794,16.0,10.0,0.0
3,18,1664541,7.0,0.0,0.0
4,19,713598,9.0,0.0,0.0
5,20,713563,60.0,20.0,1.0
6,22,475733,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,325.0,13.0,11.0
1,12,3.0,1.0,0.0
2,15,16.0,10.0,0.0
3,18,7.0,0.0,0.0
4,19,9.0,0.0,0.0
5,20,60.0,20.0,1.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,325.0
1,12,Released,3.0
2,15,Released,16.0
3,18,Released,7.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,22
Status,,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,1.0,0.0
Executing,13.0,1.0,10.0,0.0,0.0,20.0,0.0
Released,325.0,3.0,16.0,7.0,9.0,60.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,22
0,Completed,11.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,13.0,1.0,10.0,0.0,0.0,20.0,0.0
2,Released,325.0,3.0,16.0,7.0,9.0,60.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,22
0,Completed,11.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,13.0,1.0,10.0,0.0,0.0,20.0,0.0
2,Released,325.0,3.0,16.0,7.0,9.0,60.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()